In [3]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import json
import plotly.graph_objects as go

In [27]:
ticker_list = pd.read_csv('/home/yair/GHub/Codigos-em-financas/data/Cart_IBr50.csv',encoding='latin-1',
            skiprows=[0], skipfooter=2, engine='python',index_col=False,
            sep=';',decimal=',',thousands='.')['Código'].to_list()

In [ ]:
years=2
end_date = datetime.now()
start_date = end_date - timedelta(days=years * 365)
stock_data = pd.DataFrame()

for ticker in ticker_list:
    stock = yf.Ticker(ticker+'.SA')
    hist_data = stock.history(period='1d', start=start_date, end=end_date)
    close_data = hist_data['Close'].rename(ticker)
    stock_data = pd.merge(stock_data, pd.DataFrame(close_data), left_index=True, right_index=True, how='outer')

In [31]:
daily_data = stock_data.copy()
daily_data

,ALOS3,ABEV3,ASAI3,AZUL4,AZZA3,B3SA3,BBSE3,BBDC4,BBAS3,BRAV3,...,CSNA3,SUZB3,VIVT3,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,WEGE3
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-26 00:00:00-03:00,NaN,13.785151,17.634159,15.50,92.091331,12.048324,24.230219,17.122438,17.223156,NaN,...,9.498827,40.041832,36.610512,9.979613,27.381113,11.573210,6.693896,59.482281,15.825252,29.589149
2022-09-27 00:00:00-03:00,NaN,13.938619,17.673922,15.53,91.834389,11.935985,23.937483,17.018347,17.006935,NaN,...,9.329206,40.862286,36.031658,9.827897,27.875956,11.448973,6.638574,58.599380,15.268584,29.085831
2022-09-28 00:00:00-03:00,NaN,14.191392,17.842907,15.81,92.072296,12.029602,22.908722,17.027016,17.011343,NaN,...,9.414018,41.022602,36.289906,10.080756,28.341682,11.324736,6.629354,59.153355,15.348108,29.347170
2022-09-29 00:00:00-03:00,NaN,14.092089,17.842907,14.42,91.786804,11.795563,22.214520,17.157124,16.918676,NaN,...,9.282946,40.767979,35.951504,10.080756,27.720711,11.085818,6.454169,59.231262,14.853292,29.172947
2022-09-30 00:00:00-03:00,NaN,14.028895,17.445292,14.72,93.652054,12.216833,22.222883,17.209173,16.998108,NaN,...,9.807232,41.965652,36.058372,10.181902,28.477524,11.210054,6.933622,62.355976,15.242076,31.108772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-18 00:00:00-03:00,23.139999,12.980000,8.740000,5.62,47.770000,12.130000,35.590000,15.240000,28.250000,20.850000,...,11.890000,52.599998,55.810001,18.635857,29.629999,22.379999,6.260000,57.540001,24.416311,53.599998
2024-09-19 00:00:00-03:00,22.900000,12.950000,8.240000,5.61,46.299999,12.000000,35.360001,15.030000,27.910000,18.889999,...,12.050000,52.570000,55.200001,18.367643,29.790001,22.120001,6.230000,58.230000,24.257893,53.259998
2024-09-20 00:00:00-03:00,22.350000,12.760000,7.700000,5.26,44.650002,11.600000,35.009998,14.750000,27.500000,18.180000,...,11.130000,52.340000,55.279999,18.606054,28.950001,21.350000,5.880000,57.349998,24.535128,52.520000


In [32]:
def resample_data(data, period):
    if period == 'D':
        return data
    elif period == 'W':
        return data.resample('W').last()
    elif period == 'M':
        return data.resample('M').last()

In [35]:
def simulate_momentum_strategy(data, initial_amount, top_n, tax_rate = 0.15 , period='M'):
    data = resample_data(data, period)
    log_returns = np.log(data / data.shift(1))
    simulation_details = pd.DataFrame(index=log_returns.index,
                                      columns=['Portfolio', 'Profit Before Tax', 'Tax', 'Portfolio Value'])
    cash = initial_amount

    # strategy
    for i in range(0, len(log_returns) - 1):
        # top_n momentum 
        top_stocks = log_returns.iloc[i].sort_values(ascending=False).head(top_n)
        top_stocks = top_stocks[top_stocks > 0]

        if not top_stocks.empty:
            simulation_details.loc[log_returns.index[i + 1], 'Portfolio'] = json.dumps(top_stocks.index.tolist())
            num_stocks = len(top_stocks)
            allocation_per_stock = cash / num_stocks
            # Calculate new portfolio value based on the next day's returns
            new_value = sum(allocation_per_stock * np.exp(log_returns.loc[log_returns.index[i + 1], stock]) for stock in top_stocks.index)
            # Calculate and deduct tax if there is a profit
            profit = new_value - cash
            simulation_details.loc[log_returns.index[i + 1], 'Profit Before Tax'] = round(profit, 2)

            if profit > 0:
                tax = profit * tax_rate
                new_value -= tax
                simulation_details.loc[log_returns.index[i + 1], 'Tax'] = round(tax, 2)
            simulation_details.loc[log_returns.index[i + 1], 'Portfolio Value'] = round(new_value, 2)

        else:
            # No allocation, so portfolio value remains the same
            simulation_details.loc[log_returns.index[i + 1], 'Portfolio Value'] = cash
        # Update cash amount for the next round
        cash = simulation_details.loc[log_returns.index[i + 1], 'Portfolio Value']
    # Assign the initial amount to the first row
    simulation_details.loc[log_returns.index[0], 'Portfolio Value'] = initial_amount
    return simulation_details

In [37]:
# Configuration for the momentum strategy simulation
initial_amount = 100

top_n = 3

simulation_details = simulate_momentum_strategy(daily_data, initial_amount, top_n, tax_rate, frequency)

simulation_details

,Portfolio,Profit Before Tax,Tax,Portfolio Value
Date,,,,
2022-09-30 00:00:00-03:00,NaN,NaN,NaN,100
2022-10-31 00:00:00-03:00,NaN,NaN,NaN,100
2022-11-30 00:00:00-03:00,"[""PRIO3"", ""WEGE3"", ""SBSP3""]",-0.33,NaN,99.67
2022-12-31 00:00:00-03:00,"[""GGBR4"", ""CSNA3"", ""VALE3""]",0.33,0.05,99.95
2023-01-31 00:00:00-03:00,"[""BBSE3"", ""EMBR3"", ""VALE3""]",10.88,1.63,109.2
2023-02-28 00:00:00-03:00,"[""MGLU3"", ""CSNA3"", ""NTCO3""]",-8.58,NaN,100.62
2023-03-31 00:00:00-03:00,"[""MULT3"", ""NTCO3"", ""TIMS3""]",-4.3,NaN,96.32
2023-04-30 00:00:00-03:00,"[""AZUL4"", ""EMBR3"", ""CMIG4""]",-2.37,NaN,93.95
2023-05-31 00:00:00-03:00,"[""TIMS3"", ""BPAC11"", ""CPLE6""]",1.42,0.21,95.16


In [38]:
# Simulate how each individual stock would have performed over the same period
def track_individual_investments(data, initial_amount, simulation_details, period='W'):
    # Resample data based on the specified period
    data = resample_data(data, period)
    # Calculate returns based on the resampled data
    returns = data.pct_change()
    # Create a new DataFrame to store individual stock values over time
    individual_investments = pd.DataFrame(index=data.index, columns=data.columns)
    for stock in data.columns:
        # Simulate an investment in each stock
        individual_investments[stock] = (1 + returns[stock]).cumprod() * initial_amount
    # Include the Portfolio Value from the momentum strategy
    individual_investments['Portfolio Value'] = simulation_details['Portfolio Value']
    individual_investments['Baseline'] = individual_investments.iloc[:, :-1].T.mean()
    # Adjust the first values to match the Initial Amount.
    individual_investments.iloc[0, :] = initial_amount
    return individual_investments.fillna(0).astype(int)

individual_investments_df = track_individual_investments(daily_data, initial_amount, simulation_details, frequency)

individual_investments_df

,ALOS3,ABEV3,ASAI3,AZUL4,AZZA3,B3SA3,BBSE3,BBDC4,BBAS3,BRAV3,...,VIVT3,TIMS3,TOTS3,UGPA3,USIM5,VALE3,VBBR3,WEGE3,Portfolio Value,Baseline
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-30 00:00:00-03:00,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
2022-10-31 00:00:00-03:00,0,102,112,109,106,115,111,100,96,0,...,102,109,112,114,96,93,105,125,100,108
2022-11-30 00:00:00-03:00,0,102,114,83,91,97,117,78,92,0,...,93,107,105,121,103,118,95,121,99,101
2022-12-31 00:00:00-03:00,0,98,111,74,80,101,126,76,91,0,...,94,104,94,107,95,123,92,119,99,98
2023-01-31 00:00:00-03:00,0,92,112,80,89,99,142,73,107,0,...,106,99,103,112,113,131,97,119,109,103
2023-02-28 00:00:00-03:00,0,90,103,48,76,81,135,68,108,0,...,98,103,93,113,90,118,87,122,100,95
2023-03-31 00:00:00-03:00,0,97,88,81,65,80,128,69,106,0,...,98,105,96,119,96,114,85,127,96,93
2023-04-30 00:00:00-03:00,0,95,70,73,68,90,136,73,116,0,...,106,126,88,124,100,103,78,129,93,96
2023-05-31 00:00:00-03:00,0,97,61,114,75,103,122,82,121,0,...,103,126,98,145,97,90,95,118,95,102


In [40]:
from scipy.stats import ttest_1samp

def calculate_sharpe_ratio(returns, annual_risk_free_rate=0.01, frequency='D'):
    # Adjust the risk-free rate based on the frequency
    if frequency == 'D':
        adjusted_rfr = (1 + annual_risk_free_rate) ** (1/252) - 1
    elif frequency == 'W':
        adjusted_rfr = (1 + annual_risk_free_rate) ** (1/52) - 1
    elif frequency == 'M':
        adjusted_rfr = (1 + annual_risk_free_rate) ** (1/12) - 1

    excess_returns = returns - adjusted_rfr
    return excess_returns.mean() / excess_returns.std()

def t_test_portfolio_returns(portfolio_returns, bench_annual_rate=0.1, frequency='D'):
    # Adjust the risk-free rate based on the frequency
    if frequency == 'D':
        adjusted_rfr = (1 + bench_annual_rate) ** (1/252) - 1
    elif frequency == 'W':
        adjusted_rfr = (1 + bench_annual_rate) ** (1/52) - 1
    elif frequency == 'M':
        adjusted_rfr = (1 + bench_annual_rate) ** (1/12) - 1

    t_stat, p_value = ttest_1samp(portfolio_returns[1:], adjusted_rfr)  # [1:] to exclude the NaN from pct_change
    return t_stat, p_value

def calculate_metrics(dataframe, initial_amount, bench_annual_rate, frequency='D'):
    # Calculate the final and relative values
    final_values = dataframe.iloc[-1]
    relative_values = final_values / initial_amount - 1  # Subtract 1 to get the growth proportion

    # Calculate mean return and Sharpe Ratio
    returns = dataframe.pct_change()

    if frequency == 'D':
        annualization_factor = 252
    elif frequency == 'W':
        annualization_factor = 52
    elif frequency == 'M':
        annualization_factor = 12

    # Corrected annualization of mean returns
    mean_returns = (1 + returns.mean()) ** annualization_factor - 1
    sharpes = returns.apply(calculate_sharpe_ratio, annual_risk_free_rate=0.01, frequency=frequency)

    # Test if the portfolio returns are greater than the adjusted risk-free rate
    portfolio_returns = dataframe['Portfolio Value'].pct_change()
    t_stat, p_value = t_test_portfolio_returns(portfolio_returns, bench_annual_rate, frequency=frequency)

    return final_values, relative_values, mean_returns, sharpes, t_stat, p_value / 2

bench_annual_rate = 0.1

# Calculate the metrics
final_values, relative_values, mean_returns, sharpes, t_stat, p_value = calculate_metrics(individual_investments_df, initial_amount, bench_annual_rate, frequency)

In [41]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_combined_charts(dataframe, final_values, relative_values, sharpes, mean_returns):
    labels = final_values.index
    colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A']

    fig = make_subplots(rows=3, cols=2,
                        subplot_titles=('Portfolio Value Over Time',
                                        '',
                                        'Final Investment Values',
                                        'Relative Investment Growth',
                                        'Annualized Sharpe Ratios',
                                        'Annualized Mean Returns'),
                        vertical_spacing=0.08)

    # Portfolio Value line chart
    fig.add_trace(go.Scatter(x=dataframe.index,
                             y=dataframe['Portfolio Value'],
                             mode='lines',
                             name='Portfolio Value',
                             line=dict(color=colors[0], width=2.5)),
                  row=1, col=1)

    # T-test and P-value
    significance_text = f"<b>T-test:</b> {t_stat:.2f}<br><b>P-value:</b> {p_value:.5f}"
    if t_stat > 2 and p_value < 0.05:
        significance_text += f"<br><b>Significantly different from {bench_annual_rate:.0%} per year!</b>"

    fig.add_annotation(
        text=significance_text,
        showarrow=False,
        xref="x2", yref="y2",
        x=0.5, y=0.5,
        font=dict(size=15),
        bgcolor="white",
        align="center"
    )

    # Final values
    fig.add_trace(go.Bar(x=labels,
                         y=final_values.values,
                         name='Final Values ($)',
                         text=[f"${v:,.2f}" for v in final_values.values],
                         textposition='outside',
                         marker_color=colors[1]),
                  row=2, col=1)

    # Relative Growth
    fig.add_trace(go.Bar(x=labels,
                         y=relative_values.values,
                         name='Relative Growth',
                         text=[f"{v:.2%}" for v in relative_values.values],
                         textposition='outside',
                         marker_color=colors[2]),
                  row=2, col=2)

    # Sharpe Ratios
    fig.add_trace(go.Bar(x=labels,
                         y=sharpes.values,
                         name='Annualized Sharpe Ratio',
                         text=[f"{v:.2f}" for v in sharpes.values],
                         textposition='outside',
                         marker_color=colors[3]),
                  row=3, col=1)

    # Mean Returns
    fig.add_trace(go.Bar(x=labels,
                         y=mean_returns.values,
                         name='Annualized Mean Returns',
                         text=[f"{v:.2%}" for v in mean_returns.values],
                         textposition='outside',
                         marker_color=colors[4]),
                  row=3, col=2)

    # Update layout
    fig.update_layout(title_text="Investment Results Overview",
                      title_font=dict(size=24, color='black', family="Arial Black"),
                      title_pad=dict(t=10),
                      showlegend=False,
                      height=1500,
                      title_x=0.5,
                      bargap=0.05,
                      )

    fig.show()

plot_combined_charts(individual_investments_df, final_values, relative_values, sharpes, mean_returns)